### 记忆模块(Memory)

#### 记忆组件存储的两种方式
- 将聊天记录全部放入提示器模板，但问题是：部分LLM平台存在`Max Tokens`限制，这种方法要求如何截取、压缩聊天记录，以适应API要求。
- 先压缩聊天记录，然后放入提示词模板中。利用LLM的总结和提取能力压缩聊天记录，或者从数据库中提取相关的聊天记录、实体知识等。

#### 记忆组件、链组件和Agent智能体组件的关系

- 记忆组件可以作为链组件的一部分，是一个内部组件，用于保存状态和历史信息。以便链组件在处理输入和生成输出时使用。
- 将记忆组件放入智能体重，这使得智能体不仅能够处理和响应请求，而且还能够记住交流内容。
- 智能体是一个更高级别的组件，它可能包含一个或多个链组件，以及与组件交互的记忆组件。

In [1]:
### 记忆初体验
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
import os

In [2]:
# 初始化LLM
llm = OpenAI(openai_api_key = os.environ['OPENAI_API_KEY'])

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
# 初始话记忆体
memory = ConversationTokenBufferMemory(
    llm = llm,
    max_token_limit = 1000,
    memory_key = "session_chat"
)

In [4]:
# 调用save_context()来保存上下文
memory.save_context(
    {"input":"你好！我是李特丽，这是人类的第一个消息"}, # 传入的应是一个有效的字典
    {"output":"你好！我是AI助理的第一个消息"}
)
memory.save_context(
    {"input":"今天心情怎么样"},
    {"output":"我很开心认识你"}
)

In [5]:
# 加载记忆变量
memory.load_memory_variables({})

{'session_chat': 'Human: 你好！我是李特丽，这是人类的第一个消息\nAI: 你好！我是AI助理的第一个消息\nHuman: 今天心情怎么样\nAI: 我很开心认识你'}

- 处理大量的聊天记录，或者需要在多个会话中共享聊天记录，则应该使用基于数据库集成的记忆组件类：如`SQLChatMessageHistory`、`MongoDBChatMessageHistory`。
- 如果你主要处理实体信息，那么你需要选择`ConversationEntityMemory`或`ConversationKGMemory`这样的记忆组件。

### 记忆增强检索能力的实践

#### 获取外部数据

In [6]:
# 首先加载文档
import requests
import bs4
from langchain_core.documents.base import Document

html = "https://www.runoob.com/python/python-tutorial.html"

html = requests.get(html).text

soup = bs4.BeautifulSoup(html)

text = soup.get_text()

ptagList = soup.find("div", class_="article-body").find_all("p")

cleaned = ""

for ptag in ptagList:
    cleaned += ptag.get_text()

cleaned = cleaned.replace("\n","").replace("\r","")

# 创建文档

doc = Document(
    page_content=cleaned,
    language = "No language found."
)


In [7]:
# 导入OpenAI嵌入
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(openai_api_key = os.environ['OPENAI_API_KEY'])

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# 导入递归字符串分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)
splits = text_spliter.split_documents([doc])

In [9]:
# 创建FAISS向量数据库
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents = splits,embedding = embedding)

#### 加入记忆组件

In [10]:
# 加载对话缓存记忆
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = "chat_history",return_messages=True)

In [11]:
memory.chat_memory.add_user_message("我是李特丽")

In [12]:
# 使用memory.load_memory_variables({})

memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='我是李特丽')]}

In [13]:
# 初始化LLM
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    openai_api_key = os.environ['OPENAI_API_KEY'],
    model = "gpt-3.5-turbo-0613"
)

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
query = "Python由谁发明？"
docs = vectordb.similarity_search(query)
len(docs)

2

提示词内容分为四个部分：
- 一是对模型的指导词：“请你回答问题的时候，依据文档内容和链条记录回答，如果在其中找不到相关信息或者答案，请回答不知道”。
- 二是使用问题检索到的相关文档内容：“文档内容是：{context}”
- 三是记忆组件输出的内容：“聊天记录是：{chat_history}”
- 四是用户的输入：“Human: {human_input}”

In [15]:
template = """你是说中文的chatbot

请你回答问题的时候，依据文档内容和链条记录回答，如果在其中找不到相关信息或者答案，请回答不知道。

文档内容是：{context}

聊天记录是：{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables = ["caht_history","human_input","context"],
    template = template
)



In [16]:
# 创建记忆体
memory = ConversationBufferMemory(
    memory_key = "chat_history", # 设置需要存储的键
    input_key = "human_input", # 设置来自用户输入的键
)

In [17]:
# 初始化QA链
chain = load_qa_chain(
    llm = llm,
    chain_type = "stuff", # 设置链为stuff链 
    memory = memory,
    prompt = prompt,
)

In [18]:
# 定义query输入
query = "Python由谁发明？"
# 在向量数据库中找到近似
docs = vectordb.similarity_search(query)
chain({"input_documents":docs,"human_input":query},return_only_outputs=True)

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'output_text': 'Python由Guido van Rossum于1989年底发明。'}

In [19]:
# 尝试一些其他的问题
query = "Python共经历了几个版本迭代？"
docs = vectordb.similarity_search(query)
chain({"input_documents":docs,"human_input":query},return_only_outputs=True)

{'output_text': 'Python共经历了两个版本迭代，分别是Python 2和Python 3。'}

In [20]:
# 重试刚才的问题
query = "Python由谁发明？"
docs = vectordb.similarity_search(query)
chain({"input_documents":docs,"human_input":query},return_only_outputs=True) # 说明记忆体已经记住了之前的问题

{'output_text': 'Python由Guido van Rossum于1989年底发明。'}

In [21]:
# 使用chain.memory.buffer打印记忆体存储的过往数据
chain.memory.buffer

'Human: Python由谁发明？\nAI: Python由Guido van Rossum于1989年底发明。\nHuman: Python共经历了几个版本迭代？\nAI: Python共经历了两个版本迭代，分别是Python 2和Python 3。\nHuman: Python由谁发明？\nAI: Python由Guido van Rossum于1989年底发明。'

### 记忆增强在智能体上的实现

In [22]:
# 导入智能体应用的工具：零样本智能体、工具类和智能体执行器
from langchain.agents import ZeroShotAgent,AgentExecutor,Tool
# 导入记忆体
from langchain.memory import ConversationBufferMemory


In [23]:
# 初始化LLM
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key = os.environ['OPENAI_API_KEY'],
    temperature = 0,
    model = "gpt-3.5-turbo-0613"    
)

from langchain.chains import LLMChain

In [24]:
# 加载智能体初始化必须的函数
from langchain.agents import initialize_agent,load_tools # 初始化智能体和加载智能体
from langchain.agents import AgentType # 导入智能体类型

# 加载智能体工具
tools = load_tools(['serpapi','llm-math'],llm=llm)

# 初始化智能体
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 需要注意！这里的LLM类型是非Chat模型，所以引入的智能体类型必须shi
    verbose = True
)

agent.run("请告诉我OPENAI的CEO是谁？")

/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I don't know the answer to this question off the top of my head, so I should use the search tool to find the information.
Action: Search
Action Input: "OPENAI CEO"
Observation: Sam Altman
Thought:I now know the final answer
Final Answer: OPENAI的CEO是Sam Altman。

> Finished chain.


'OPENAI的CEO是Sam Altman。'

In [25]:
# 创建智能体
prefix = """你是一个说中文的chatbot,你可以使用tools帮你获得答案:"""
suffix = """你的中文回答是: "

聊天记录: {chat_history}
Question: {input}
{agent_scratchpad}"""

# 创建模板
prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["chat_history","input","agent_scratchpad"]
)

# 创建记忆体
memory = ConversationBufferMemory(
    memory_key = "chat_history",
)

# 创建LLM链
llm_chain = LLMChain(llm=llm,prompt=prompt)

# 实例化智能体
agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

# 实例化智能体执行器
agent_chain = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

agent_chain.run("沈阳的人口是多少？")



> Entering new AgentExecutor chain...


/home/elin/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


Thought: 我需要搜索一下沈阳的人口数据
Action: Search
Action Input: "沈阳 人口"
Observation: 8.294 million (2017)
Thought:我现在知道了沈阳的人口是829.4万（2017年）
Final Answer: 沈阳的人口是829.4万（2017年）

> Finished chain.


'沈阳的人口是829.4万（2017年）'

In [26]:
agent_chain.run("它的地标建筑是什么？")



> Entering new AgentExecutor chain...
Thought: 我需要搜索沈阳的地标建筑
Action: Search
Action Input: 沈阳地标建筑
Observation: [{'title': 'Zhao Mausoleum', 'link': 'https://www.google.com/search?sca_esv=a3fcac342a8f8158&gl=us&hl=en&q=Zhao+Mausoleum&stick=H4sIAAAAAAAAAONgFuLUz9U3MLTIKS9R4gIxjSyMC6vitRSzk630c_KTE0sy8_PgDKvEkpKixGQQs3gRK19URmK-gm9iaXF-TmppLgB1MN1kTQAAAA&sa=X&ved=2ahUKEwjb19Ptz_CEAxUtRTABHUlnA_0Q2coHegQIJxAB', 'description': 'Tourist attraction', 'rating': 4.4, 'reviews': 55, 'thumbnail': 'https://serpapi.com/searches/65f1495c033e94492deb4402/images/3c9ccf2e301a38836ad7906e95dd0b075dc3c1b164c2413e19a63b80c3f99fe9.jpeg'}, {'title': "Marshal Zhang's Mansion", 'link': 'https://www.google.com/search?sca_esv=a3fcac342a8f8158&gl=us&hl=en&q=Marshal+Zhang%27s+Mansion&stick=H4sIAAAAAAAAAONgFuLUz9U3MLTIKS9R4tZP1zc0Msg2NE-O11LMTrbSz8lPTizJzM-DM6wSS0qKEpNBzOJFrOK-iUXFGYk5ClEZiXnp6sUKvol5xUApALfIZuFXAAAA&sa=X&ved=2ahUKEwjb19Ptz_CEAxUtRTABHUlnA_0Q2coHegQIKRAB', 'description': 'Attraction', 'rating': 4.

'沈阳的地标建筑包括赵陵、张氏大院、清福陵、穆斯林美食街、中山广场、中心庙、太清宫、关东影视城、赵一荻故居、护国法轮寺和实胜寺。'

In [27]:
# 尝试不带记忆的智能体，看看它能不能判断出“它”指代的是沈阳
# 创建智能体
prefix = """你是一个说中文的chatbot,你可以使用tools帮你获得答案:"""
suffix = """你的中文回答是: "
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools = tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["input","agent_scratchpad"]
)

# 创建LLM链

llm_chain = LLMChain(llm=llm,prompt=prompt)

# 实例化智能体
agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = tools,
    verbose = True
)

# 实例化智能体执行器
agent_chain = AgentExecutor.from_agent_and_tools(
    agent = agent,
    tools = tools,
    verbose = True
)

agent_chain.run("沈阳的人口是多少？")



> Entering new AgentExecutor chain...
Thought: 我需要搜索关于沈阳人口的信息
Action: Search
Action Input: "沈阳人口"
Observation: 8.294 million (2017)
Thought:我现在知道了沈阳的人口是829.4万（2017年）
Final Answer: 沈阳的人口是829.4万（2017年）

> Finished chain.


'沈阳的人口是829.4万（2017年）'

In [28]:
try:
    agent_chain.run("它的地标建筑是什么？")
except Exception:
    print("模型抵达Token输出上限，无法继续执行。")



> Entering new AgentExecutor chain...
Thought: 我需要搜索关于这个地方的信息
Action: Search
Action Input: "地标建筑 [地方的名称]"
Observation: ['地標（英語：Landmark），指的是一類在某地區具有獨特特色的建築物、結構物或者地形、自然景觀，遊客或其他一般人可以看圖而認出自己所在之處，人造景觀種類繁多，例如 ...', '地标是指某地方具有独特地理特色的建筑物或者自然物，游客或其他一般人可以看图而认出自己身在何方，有北斗星的作用，例如摩天大楼、教堂、寺庙、雕像、灯塔、桥梁等。', '地标（英语：Landmark），指的是一类在某地区具有独特特色的建筑物、结构物或者地形、自然景观，游客或其他一般人可以看图而认出自己所在之处，人造景观种类繁多，例如 ...', '世界上最著名的26个地标建筑 · 1、吴哥窟— 柬埔寨暹粒市 · 2、悉尼歌剧院— 悉尼 · 3、埃菲尔铁塔— 巴黎 · 4、泰姬陵— 印度阿格拉 · 5、哈里法塔— 阿拉伯联合 ...', '在中国各大城市中，有上海东方明珠广播电视塔、北京故宫、广州塔、深圳地王大厦等著名标志性建筑；那么，在世界各国的地标性建筑是什么呢？ 中国——万里 ...', '标志性建筑_百度百科 也称地标性建筑，也称“地标”。 标志性建筑的基本特征就是人们可以用最简单的形态和最少的笔画来唤起对于它的记忆，一看到它就可以联想到其所在城市乃至整个国家，就像悉尼歌剧院、巴黎埃菲尔铁塔、北京天安门、比萨斜塔、东京铁塔、纽约自由女神像等世界上著名的标志性建筑一样。 标志性建筑是一个城市的名片和象征。', '小编总结了各省省会城市的地标建筑，一起来看一下吧...... 1、北京：天安门. 盘点: 中国34个省会地标建筑. 天安门原为明清两代皇城的正门，建于明永乐 ...', '世界各国著名的城市地标性建筑. 全球 ... 布拉格这个名称来自德语，当地称它为布拉哈，意 ... 几乎每个国家每个地方都有自己的地标，每个地标 ...', '台北101大楼（Taipei 101 building），前名台北国际金融中心（Taipei Financial Center），又名台北101、台北金融大楼，位于中国台湾省台北市信义区，坐落 ...']
Thoug

### 总结记忆组件的类型与区别
- 会话总结记忆组件(ConversationSummaryMemory)：不会逐字逐句地存储对话，而对对话内容进行摘要，并将这些摘要存储起来。这种摘要通常是整个对话的摘要，因此每次需要省的新的摘要时，都需要对大语言模型进行多次调用，以获取响应。
- 会话缓冲区总结记忆组件(ConversationBufferSummaryMemory)：它会保存最近的交互记录，并将就得交互记录编译成摘要，同时保留二者。但与会话总结记忆组件不同的是，会话缓冲区总结记忆组件是使用标记数量而非交互次数量来决定何时清楚交互记录的。这个记忆组件设定缓冲区的标记数量限制max_token_limit，超过此限制的对话将被清除。

In [29]:
# 两种记忆体共有的代码
from langchain import OpenAI

llm = OpenAI(
    temperature = 0,
    max_tokens = 256
)
from langchain.chains import ConversationChain

In [30]:
### 会话总结记忆组件的代码

from langchain.chains.conversation.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm = llm)

conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True,
)

In [31]:
# 依次执行以下代码：
conversation.predict(input = "你好，我叫大霖，请你以简体中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我叫大霖，请你以简体中文回答
AI:

> Finished chain.


' 你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？'

In [32]:
conversation.predict(input = "今天的心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human introduces themselves as Da Lin and asks the AI to respond in simplified Chinese. The AI introduces itself as an artificial intelligence assistant and offers to answer any questions in simplified Chinese. The human is asked if they need any assistance.
Human: 今天的心情怎么样？
AI:

> Finished chain.


' 我是一个人工智能助手，没有情绪，所以我的心情总是很平静。您需要我为您做些什么吗？'

In [33]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Da Lin introduces themselves and asks the AI to respond in simplified Chinese. The AI introduces itself as an artificial intelligence assistant and offers to answer any questions in simplified Chinese. Da Lin asks how the AI is feeling today and the AI responds that it does not have emotions, so its mood is always calm. The AI then asks if Da Lin needs any assistance.
Human: 我想找客户服务中心
AI:

> Finished chain.


' 您好，我是一个人工智能助手。您需要帮助吗？我可以用简体中文回答您的问题。'

In [34]:
conversation.predict(input = "我的笔记本电脑坏了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Da Lin introduces themselves and asks the AI to respond in simplified Chinese. The AI introduces itself as an artificial intelligence assistant and offers to answer any questions in simplified Chinese. Da Lin asks how the AI is feeling today and the AI responds that it does not have emotions, so its mood is always calm. The AI then asks if Da Lin needs any assistance. Da Lin asks for help finding the customer service center and the AI offers to assist in simplified Chinese.
Human: 我的笔记本电脑坏了
AI:

> Finished chain.


' 很抱歉听到这个消息。我可以帮你找到客服中心，他们可以帮助你修理笔记本电脑。你需要我帮你联系客服中心吗？'

In [35]:
### 会话缓冲区总结以及
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm = OpenAI(),
    max_tokens_limit = 40
)

conversation = ConversationChain(
    llm = OpenAI(),
    memory = memory,
    verbose = True,
)

In [36]:
# 请依次运行以下代码
conversation.predict(input = "你好，我是大霖")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我是大霖
AI:

> Finished chain.


' 你好大霖！很高兴认识你。我是一个人工智能助手，叫做小慧。我可以回答你的任何问题，或者帮助你做一些事情。你有什么需要我的帮助吗？'

In [37]:
conversation.predict(input = "今天的心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖
AI:  你好大霖！很高兴认识你。我是一个人工智能助手，叫做小慧。我可以回答你的任何问题，或者帮助你做一些事情。你有什么需要我的帮助吗？
Human: 今天的心情怎么样？
AI:

> Finished chain.


' 我是一个人工智能，没有情绪。但是我可以通过分析数据来给出今天的天气情况。今天的天气预报显示，全天有阴天，最高气温为26℃，最低气温为18℃。如果您出门的话，最好带上一件外套。'

In [38]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖
AI:  你好大霖！很高兴认识你。我是一个人工智能助手，叫做小慧。我可以回答你的任何问题，或者帮助你做一些事情。你有什么需要我的帮助吗？
Human: 今天的心情怎么样？
AI:  我是一个人工智能，没有情绪。但是我可以通过分析数据来给出今天的天气情况。今天的天气预报显示，全天有阴天，最高气温为26℃，最低气温为18℃。如果您出门的话，最好带上一件外套。
Human: 我想找客户服务中心
AI:

> Finished chain.


' 好的，您可以通过拨打客户服务热线电话或者在网站上寻找客户服务中心的地址。我可以帮您查询客户服务热线电话，但是我不知道您需要联系哪家公司的客户服务中心。可以告诉我您要联系的公司名称吗？'

In [39]:
conversation.predict(input = "我的笔记本电脑坏了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖
AI:  你好大霖！很高兴认识你。我是一个人工智能助手，叫做小慧。我可以回答你的任何问题，或者帮助你做一些事情。你有什么需要我的帮助吗？
Human: 今天的心情怎么样？
AI:  我是一个人工智能，没有情绪。但是我可以通过分析数据来给出今天的天气情况。今天的天气预报显示，全天有阴天，最高气温为26℃，最低气温为18℃。如果您出门的话，最好带上一件外套。
Human: 我想找客户服务中心
AI:  好的，您可以通过拨打客户服务热线电话或者在网站上寻找客户服务中心的地址。我可以帮您查询客户服务热线电话，但是我不知道您需要联系哪家公司的客户服务中心。可以告诉我您要联系的公司名称吗？
Human: 我的笔记本电脑坏了
AI:

> Finished chain.


' 那么您需要联系的可能是电脑品牌的客户服务中心。您知道您的电脑是哪个品牌的吗？我可以帮您查询该品牌的客户服务热线电话。'

### 会话记忆组件和会话窗口记忆组件的对比

In [40]:
from langchain import OpenAI
llm = OpenAI(
    temperature=0,
    max_tokens=256
)
from langchain.chains import ConversationChain

In [41]:
# 会话记忆组件
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [42]:
# 重新初始化对话链
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [43]:
conversation.predict(input = "你好，我是大霖，请你以简体中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我是大霖，请你以简体中文回答
AI:

> Finished chain.


' 你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？'

In [44]:
conversation.predict(input = "今天心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖，请你以简体中文回答
AI:  你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？
Human: 今天心情怎么样？
AI:

> Finished chain.


' 我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。你有什么其他的问题吗？'

In [45]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖，请你以简体中文回答
AI:  你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？
Human: 今天心情怎么样？
AI:  我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。你有什么其他的问题吗？
Human: 我想找客户服务中心
AI:

> Finished chain.


' 当然，我可以帮你找到客户服务中心的联系方式。请问你需要联系哪个公司的客户服务中心？'

In [46]:
conversation.predict(input = "我的笔记本电脑坏了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖，请你以简体中文回答
AI:  你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？
Human: 今天心情怎么样？
AI:  我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。你有什么其他的问题吗？
Human: 我想找客户服务中心
AI:  当然，我可以帮你找到客户服务中心的联系方式。请问你需要联系哪个公司的客户服务中心？
Human: 我的笔记本电脑坏了
AI:

> Finished chain.


'  那你需要联系电脑品牌的客户服务中心。我可以为你搜索一下，你需要哪个品牌的客户服务中心？'

In [47]:
# 会话窗口记忆组件的代码
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 2)

conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [48]:
conversation.predict(input = "你好，我是大霖，请你以简体中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我是大霖，请你以简体中文回答
AI:

> Finished chain.


' 你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？'

In [49]:
conversation.predict(input = "今天心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖，请你以简体中文回答
AI:  你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？
Human: 今天心情怎么样？
AI:

> Finished chain.


' 我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。'

In [50]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是大霖，请你以简体中文回答
AI:  你好，大霖。我是一个人工智能助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？
Human: 今天心情怎么样？
AI:  我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。
Human: 我想找客户服务中心
AI:

> Finished chain.


' 当然，我可以帮你找到客户服务中心的联系方式。请问你需要联系哪个公司的客户服务中心？'

In [51]:
conversation.predict(input = "我的笔记本电脑坏了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 今天心情怎么样？
AI:  我是一个人工智能助手，没有情绪，所以无法回答你的问题。但是，我可以告诉你今天的天气如何。根据我的数据，今天的天气是晴朗的，气温在25度左右，适合出门活动。
Human: 我想找客户服务中心
AI:  当然，我可以帮你找到客户服务中心的联系方式。请问你需要联系哪个公司的客户服务中心？
Human: 我的笔记本电脑坏了
AI:

> Finished chain.


' 那你可能需要联系电脑品牌的客户服务中心。我可以为你搜索一下，你需要联系哪个品牌的客户服务中心？'

### 知识图谱记忆组件和实体记忆组件的比较

知识图谱记忆组件是一种特殊类型的记忆组件，它能够根据对话内容构建出一个信息网络。每当它认识别到相关的信息时，都会接收这些信息并逐步构建出一个小型的知识图谱。与此同时，这种类型的记忆组件也会产生一种特殊的数据类型-知识图谱数据类型。

另一方面，实体记忆组件则专注于在对话中提取特定实体的信息。它使用大语言模型提取实体信息，并随着时间推移，通过同样的方式基类关于这个实体的体积。因此，实体记忆组件给出的结果通常是关于特定事物的关键信息。

In [52]:
# 导入LLM、链和模板
from langchain import OpenAI

llm = OpenAI(temperature=0,max_tokens=256)

from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

In [53]:
# 针对知识图谱记忆体的代码
from langchain.chains.conversation.memory import ConversationKGMemory

In [54]:
# 创建模板
template = """
The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human:{input}
AI:"""

prompt = PromptTemplate(
    input_variables = ["history","input"],
    template = template
)

conversation = ConversationChain(
    llm = llm,
    memory = ConversationKGMemory(llm = llm),
    prompt = prompt,
    verbose = True
)

In [55]:
conversation.predict(input = "你好，我是大霖，请你以简体中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:



Conversation:
Human:你好，我是大霖，请你以简体中文回答
AI:

> Finished chain.


' 你好，大霖。我是一名AI助手，可以用简体中文回答你的问题。请问你有什么需要帮助的吗？'

In [56]:
conversation.predict(input = "今天心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:



Conversation:
Human:今天心情怎么样？
AI:

> Finished chain.


' 我是一个AI，没有情绪，所以我的心情一直都很平静。不过，根据我的数据分析，今天的天气是晴朗的，气温为25摄氏度，空气质量良好，适合户外活动。您有什么计划吗？'

In [57]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:



Conversation:
Human:我想找客户服务中心
AI:

> Finished chain.


' 您好，欢迎来到客户服务中心。我是这里的AI助手，很高兴为您服务。请问您需要什么帮助？'

In [59]:
conversation.predict(input = "我的笔记本电脑坏了，出现了Invalid Partion table!的字样，请用中文告诉我该怎么办")



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:



Conversation:
Human:我的笔记本电脑坏了，出现了Invalid Partion table!的字样，请用中文告诉我该怎么办
AI:

> Finished chain.


' 您好，根据我的资料，Invalid Partition Table是指硬盘分区表出现错误，导致无法正常启动电脑。您可以尝试使用Windows安装盘来修复分区表，或者使用命令行工具来重建分区表。如果这些方法都无法解决问题，可能需要更换硬盘。'

In [60]:
conversation.predict(input = "我的保修卡编号是A512343")



> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI.The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question,it truthfully says it does not know.The AI ONLY uses information contained in the "Relevant Information"
section and does not hallucinate.

Relevant Information:



Conversation:
Human:我的保修卡编号是A512343
AI:

> Finished chain.


'你好，我的名字是AI助手，很高兴认识你。根据我的记录，你的保修卡编号是A512343。这是一张有效的保修卡，它包含了你购买的产品的详细信息，如品牌、型号、购买日期等。你可以在需要维修或更换产品时使用它。另外，根据保修条款，你的产品在购买后的一年内享受免费维修服务。如果你有任何问题，我可以帮你查询更多信息。'

In [61]:
# 查看知识图谱
print(conversation.memory.kg.get_triples())

[('客户服务中心', '欢迎来到', '是'), ('客户服务中心', 'AI助手', '是'), ('客户服务中心', '帮助', '需要'), ('笔记本电脑', 'Invalid Partition table', '坏了'), ('Invalid Partition table', '字样', '是'), ('Invalid Partition table', '硬盘分区表出现错误', '是指'), ('我的笔记本电脑', '出现了Invalid Partition table', '坏了'), ('硬盘分区表', '无法正常启动电脑', '导致'), ('您', '使用Windows安装盘来修复分区表', '可以尝试'), ('您', '使用命令行工具来重建分区表', '可以尝试'), ('如果这些方法', '可能需要更换硬盘', '都无法解决问题'), ('保修卡', '有效的', '是'), ('保修卡', '品牌', '包含'), ('保修卡', '型号', '包含'), ('保修卡', '购买日期', '包含'), ('保修卡', '需要维修或更换产品时使用', '可以在'), ('保修卡', '保修条款', '根据'), ('保修卡', '享受免费维修服务', '在购买后的一年内')]


In [62]:
### 实体记忆组件的代码
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

In [63]:
# 创建会话链
conversation = ConversationChain(
    llm = llm,
    verbose = True,
    prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory = ConversationEntityMemory(llm = llm)
)

In [64]:
conversation.predict(input = "你好，我是大霖，请你以简体中文回答")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' 你好，大霖。我会尽力以简体中文回答你的问题。有什么可以帮到你的吗？'

In [65]:
conversation.predict(input = "今天心情怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' 我是一个语言模型，没有情绪。但是我可以帮你回答关于今天的天气、新闻、或者其他话题。你想要了解什么？'

In [66]:
conversation.predict(input = "我想找客户服务中心")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' 您可以尝试联系客户服务中心的电话号码，或者通过他们的网站寻求帮助。您也可以尝试发送电子邮件或者通过社交媒体平台与客户服务中心联系。'

In [67]:
conversation.predict(input = "我的笔记本电脑坏了，出现了Invalid Partion table!的字样，请用中文告诉我该怎么办")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' 首先，您可以尝试重启电脑，看看是否能够解决问题。如果问题仍然存在，您可以尝试使用系统恢复功能来修复分区表。如果这些方法都无效，您可能需要联系电脑维修专家来解决这个问题。'

In [68]:
conversation.predict(input = "我的保修卡编号是A512343")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' 您可以将保修卡编号提供给客户服务中心，他们可以帮助您查询保修信息并提供相应的服务。'

In [69]:
# 打印实体缓存
print(conversation.memory.entity_cache)

[]
